# **SML Project: Binary Tree Predictors**

*   **Author:** Matteo Onger
*   **Date:** October 2024

**Dataset documentation**:
*   [Secondary Mushroom](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset)

## VM Setup

In [ ]:
# install dataset package
!pip install ucimlrepo

# download repository
!git clone -b dev https://github.com/MatteoOnger/SML_Project.git

# set working directory
%cd /content/SML_Project/

## Code

In [2]:
# ---- LIBRARIES ----
import logging

from ucimlrepo import fetch_ucirepo

from bintreepredictor import BinTreePredictor
from data import DataSet

In [3]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", force=True)

In [4]:
mushroom = fetch_ucirepo(id=848)
mushroom_df = mushroom.data.original

In [5]:
mushroom_df = mushroom_df.drop(columns=mushroom_df.columns[mushroom_df.isna().any()])

In [6]:
train_df = mushroom_df.sample(frac=0.8, random_state=0)
test_df = mushroom_df.drop(train_df.index)

train_ds = DataSet(train_df, "class")
test_ds = DataSet(test_df, "class")

In [ ]:
tree = BinTreePredictor("mode", "gini", "max_nodes", 500, max_thresholds=5)
tree.fit(train_ds)

In [ ]:
_, accuracy = tree.predict(test_ds)
accuracy